In [1]:
import pandas as pd
import numpy as np

In [2]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def country_name_to_continent_code(country_name):
    '''Takes a country name and gives back the code of the continent the country is located in.'''
    
    #Manually assigning Russia to Asia
    if(country_name in ["Russia", "russia"]):
        return "AS"
    
    else:
        #try to obtain a valid continent code, else return NaN
        try:
            country_code = country_name_to_country_alpha2(country_name)
            continent = country_alpha2_to_continent_code(country_code)
            return continent
        except:
            return np.nan   

In [3]:
#Path to all data
path = "./data/"

# Import Airport Dataset

In [4]:
#Filename of the file containing details of the airport
fileAirports = "airports.csv"

#Import the airport dataset
dtypes = {"Name": str, "City": str, "Country": str, "IATA": str, "ICAO": str, "Latitude": np.float32, "Longitude": np.float32}
usecols = list(dtypes.keys())
airports = pd.read_csv(path + fileAirports, usecols=usecols, dtype = dtypes, na_values="\\N")

#Get Continent Code from country name
airports["continent"] = airports.Country.apply(country_name_to_continent_code)

airports.rename(columns = {"Name": "airport_name", "City": "city", "Country": "country", "Latitude": "latitude", "Longitude": "longitude"}, inplace = True)

In [11]:
airports.to_csv(path + "airports_clean.csv")

# Define Cleaning Function

In [5]:
def get_clean_dataset(df, continents = ["EU"], domestic_flights_only = True):

    '''Cleans a given flight dataset for a given continent.
    NaNs in origin and destination IATA Codes are dropped in order to (1) remove incomplete information and 
    (2) remove flights from or to airports without an IATA code (regional airports).
    Flights origined and destined at the same airport are dropped, since assumed to be non commercial.
    Flights are aggregated by origin and destination with the number of unique callsigns as their weight.
    Finally additional information about origin and destination airport is added.
    
    If domestic_flights_only == False the function also returns fligths either origined or destined in other continents.'''

    try:
        df = pd.DataFrame(df)
    except:
        raise TypeError("df must be specified and convertable to a DataFrame.")
    
    #adding IATA Code and Continent to the dataset
    for dir in ["origin", "destination"]:
        df = df.merge(airports[["ICAO", "IATA", "continent"]].add_prefix(dir + "_"), 
                            how = "left", left_on = dir, right_on = dir + "_ICAO", validate = "m:1")

    #dropping NaNs in IATA Codes
    df.dropna(subset=["origin_IATA", "destination_IATA"], inplace=True)

    #dropping rows where origin == destination
    df = df.loc[df["origin_IATA"] != df["destination_IATA"]]

    #filtering for the specified continent, if domestic_flights_only is False, flights origined or destined in other continents are considered too
    if domestic_flights_only:
        df = df.loc[(df.origin_continent.isin(continents)) & (df.destination_continent.isin(continents))]
    else:
        df = df.loc[(df.origin_continent.isin(continents)) | (df.destination_continent.isin(continents))]
    

    #grouping of the airports by origin and destination
    df = df.groupby(["origin_IATA", "destination_IATA"]).nunique().reset_index()

    #delete not needed columns
    df = df \
    .drop(columns = ["origin", "destination", "origin_ICAO", "origin_continent", "destination_ICAO", "destination_continent"]) \
    .rename(columns = {"callsign": "weight", "icao24": "rec_aircrafts", "day": "rec_days"})

    #delete flights that only occur once in a month
    df = df.loc[df.rec_days>1]

    #add additional information about the airports
    for dir in ["origin", "destination"]:
        df = df.merge(airports.add_prefix(dir + "_"), 
                            how = "left", on = dir + "_IATA")

    return df

# Import, Clean and Save Flight Data

In [6]:

file2022 = "flightlist_20220801_20220830.csv.gz"
file2019 = "flightlist_20190801_20190831.csv.gz"
usecols = ["callsign", "icao24", "origin", "destination", "day"]
database2022 = pd.read_csv(path + file2022, usecols = usecols, compression = "gzip", parse_dates = ["day"])
database2019 = pd.read_csv(path + file2019, usecols = usecols, compression = "gzip", parse_dates = ["day"])

In [18]:
continents = ["EU"]
clean_db2022 = get_clean_dataset(database2022, continents = continents)
clean_db2019 = get_clean_dataset(database2019, continents = continents)

In [19]:
#Replace the outdated IATA Code of SXF with the new BER for 2022
clean_db2022.loc[clean_db2022.origin_IATA == "SXF", "origin_IATA"] = "BER"
clean_db2022.loc[clean_db2022.destination_IATA == "SXF", "destination_IATA"] = "BER"

In [20]:
filename2019 = "-".join(continents) + "_routes_2019.csv.gz"
filename2022 = "-".join(continents) + "_routes_2022.csv.gz"
clean_db2019.to_csv(path + filename2019, compression = "gzip", index = False)
clean_db2022.to_csv(path + filename2022, compression = "gzip", index = False)